In [2]:
#!pip install tweet-preprocessor
import pandas as pd
from commen_preprocess import *
import preprocessor as p
from tqdm import tqdm
%load_ext jupyternotify
eng_train_dataset = pd.read_csv('../Data/english_dataset/english_dataset.tsv', sep='\t')
hindi_train_dataset = pd.read_csv('../Data/hindi_dataset/hindi_dataset.tsv', sep='\t')
german_train_dataset = pd.read_csv('../Data/german_dataset/german_dataset.tsv', sep='\t')


.....start_cleaning.........
<hashtag> britain exit <hashtag> rape refugee


<IPython.core.display.Javascript object>

In [3]:
german_train_dataset.head()

,text_id,text,task_1,task_2
0,hasoc_de_1,Frank Rennicke – Ich bin stolz https://t.co/Cm...,NOT,NONE
1,hasoc_de_2,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,NOT,NONE
2,hasoc_de_3,#Koeln Mohamed erkennt kein deutsches Recht so...,NOT,NONE
3,hasoc_de_4,#SaudiArabien ist eine brutale islamische Dikt...,NOT,NONE
4,hasoc_de_5,Bundespolizei #München hat im 1. Quartal 2019 ...,NOT,NONE


In [24]:
#### commen means our method and lib means to use the library tokenizer
def task1_save_two_preprocessing_files(path_to_folder,dataset,type_of_language):
    folder_to_commen = path_to_folder+'_commen.txt'
    folder_to_lib = path_to_folder+'_lib.txt'
    with open(folder_to_commen,'w') as f:
        for i in range(len(dataset['text'])):
            if(type_of_language=="hindi"):
                f.write(clean_hindi(dataset['text'][i])+"\n")
            else:
                f.write(clean_english(dataset['text'][i])+"\n")
            
    with open(folder_to_lib,'w') as f:
        for i in range(len(dataset['text'])):
            f.write(p.clean(dataset['text'][i])+"\n")
        

####task1    
save_two_preprocessing_files("../Data/english_dataset/english_data_task1",eng_train_dataset,type_of_language="english")
save_two_preprocessing_files("../Data/german_dataset/german_data_task1",german_train_dataset,type_of_language="german")
save_two_preprocessing_files("../Data/hindi_dataset/hindi_data_task1",hindi_train_dataset,type_of_language="hindi")

eng_train_dataset_HOF = eng_train_dataset.loc[eng_train_dataset['task_1'] == 'HOF'].reset_index()
german_train_dataset_HOF = german_train_dataset.loc[german_train_dataset['task_1'] == 'HOF'].reset_index()
hindi_train_dataset_HOF = hindi_train_dataset.loc[hindi_train_dataset['task_1'] == 'HOF'].reset_index()

####task2 and 3  
save_two_preprocessing_files("../Data/english_dataset/english_data_task23",eng_train_dataset_HOF,type_of_language="english")
save_two_preprocessing_files("../Data/german_dataset/german_data_task23",german_train_dataset_HOF,type_of_language="german")
save_two_preprocessing_files("../Data/hindi_dataset/hindi_data_task23",hindi_train_dataset_HOF,type_of_language="hindi")






In [25]:
#bert-base-multilingual-casimport torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
#from pytorch_tranformers import BertTokenizer, BertModel, BertForMaskedLM
#from pytorch_transformers import *
import torch
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')



The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [26]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias

In [21]:
def get_bert_embeddings(df,type_of_language="hindi"):
        data_df=list(df.text.values)
        all_sentence_embeddings = []
        for data in tqdm(data_df):
            if(type_of_language=="hindi"):
                marked_text = "[CLS] " + clean_hindi(data) + " [SEP]"
            else:
                marked_text = "[CLS] " + clean_english(data) + " [SEP]"
            tokenized_text = tokenizer.tokenize(marked_text)
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
            segments_ids = [1] * len(tokenized_text)
            tokens_tensor = torch.tensor([indexed_tokens])
            segments_tensors = torch.tensor([segments_ids])
            with torch.no_grad():
                encoded_layers, _ = model(tokens_tensor, segments_tensors)
            sentence_embedding = torch.mean(encoded_layers[11], 1)
            sentence_embedding = sentence_embedding.data.numpy()
            all_sentence_embeddings.append(sentence_embedding[0])
        
        return all_sentence_embeddings

In [28]:
import pickle
eng_sentence_embeddings=get_bert_embeddings(eng_train_dataset,type_of_language="english")
ger_sentence_embeddings=get_bert_embeddings(german_train_dataset,type_of_language="german")
hin_sentence_embeddings=get_bert_embeddings(hindi_train_dataset,type_of_language="hindi")


filename = '../Data/english_dataset/no_preprocess_bert_embed_task1.pkl'
outfile = open(filename,'wb')
pickle.dump(eng_sentence_embeddings,outfile)
outfile.close()


filename = '../Data/german_dataset/no_preprocess_bert_embed_task1.pkl'
outfile = open(filename,'wb')
pickle.dump(ger_sentence_embeddings,outfile)
outfile.close()

filename = '../Data/hindi_dataset/no_preprocess_bert_embed_task1.pkl'
outfile = open(filename,'wb')
pickle.dump(hin_sentence_embeddings,outfile)
outfile.close()


100%|██████████| 4665/4665 [04:29<00:00, 17.28it/s]


In [29]:
import pickle
eng_sentence_embeddings=get_bert_embeddings(eng_train_dataset_HOF,type_of_language="english")
ger_sentence_embeddings=get_bert_embeddings(german_train_dataset_HOF,type_of_language="german")
hin_sentence_embeddings=get_bert_embeddings(hindi_train_dataset_HOF,type_of_language="hindi")


filename = '../Data/english_dataset/no_preprocess_bert_embed_task23.pkl'
outfile = open(filename,'wb')
pickle.dump(eng_sentence_embeddings,outfile)
outfile.close()


filename = '../Data/german_dataset/no_preprocess_bert_embed_task23.pkl'
outfile = open(filename,'wb')
pickle.dump(ger_sentence_embeddings,outfile)
outfile.close()

filename = '../Data/hindi_dataset/no_preprocess_bert_embed_task23.pkl'
outfile = open(filename,'wb')
pickle.dump(hin_sentence_embeddings,outfile)
outfile.close()


100%|██████████| 2469/2469 [02:18<00:00, 17.77it/s]


In [ ]:
clean_english(german_train_dataset[1][50])

In [ ]:
clean_hindi(hindi_train_dataset[1][11])

In [ ]:
count=0
all_emojis=[]
for i in range(len(german_train_dataset[1])):
    parsed_tweet = p.parse(german_train_dataset[1][i])
    if parsed_tweet.mentions is not None:
        for i in range(len(parsed_tweet.mentions)):
            all_emojis.append(parsed_tweet.mentions[i].match)
        count+=1
print(count)

In [ ]:
print(parsed_tweet.emojis)

In [ ]:
import matplotlib.pyplot as plt

uniq_emojis=list(set(all_emojis))
histo = []

for i in uniq_emojis:
    histo.append(all_emojis.count(i)) ## add the number of occurances to the histo list

plt.bar(uniq_emojis, histo)
plt.show()

In [ ]:
len(uniq_emojis)

In [ ]:






text = "Here is the sentence I want embeddings for."
text = "After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank."
marked_text = "[CLS] " + text + " [SEP]"

print (marked_text)

In [ ]:
tokenized_text = tokenizer.tokenize(marked_text)
print (tokenized_text)

In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

for tup in zip(tokenized_text, indexed_tokens):
  print (tup)

In [ ]:
segments_ids = [1] * len(tokenized_text)
print (segments_ids)

In [ ]:
%%notify
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

In [ ]:
# Convert the hidden state embeddings into single token vectors

# Holds the list of 12 layer embeddings for each token
# Will have the shape: [# tokens, # layers, # features]
token_embeddings = [] 

# For each token in the sentence...
for token_i in range(len(tokenized_text)):
  
  # Holds 12 layers of hidden states for each token 
  hidden_layers = [] 
  
  # For each of the 12 layers...
  for layer_i in range(len(encoded_layers)):
    
    # Lookup the vector for `token_i` in `layer_i`
    vec = encoded_layers[layer_i][batch_i][token_i]
    
    hidden_layers.append(vec)
    
  token_embeddings.append(hidden_layers)

# Sanity check the dimensions:
print ("Number of tokens in sequence:", len(token_embeddings))
print ("Number of layers per token:", len(token_embeddings[0]))

In [ ]:
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# For each token in the sentence...
for token in token_embeddings:
    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), 0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

In [ ]:
sentence_embedding = torch.mean(encoded_layers[11], 1)

In [ ]:
sentence_embedding.data.numpy()